## ANN High-Level Diagram

```
           f_1(X)      f_2(X)             f_N(X)
          +------+    +------+           +------+  output y
input     |      | -> |      | -> ... -> |      | ---------->        comparison
--------> |      |    |      |           |      |                vs. expected output
          |      | <- |      | <- ... <- |      | <----------         E(y, y')
          +------+    +------+           +------+   error Z
           b_1(Z)      b_2(Z)             b_N(Z)
```

## Setting up the network

We need to choose a few hyperparameters:

- number of layers $N$
- input and output size of each layer $(n_1, m_1), \dots , (n_N, m_N)$ such that $m_i = n_{i+1}, i \in [1, N-1]$.
- activation function for each layer
- optimizer function (if using a library which supports it)
- learning rate $l$
- loss function $E$

Weights are initialized randomly or set to 0 at the beginning of training.

## Forward Propagation

We can define a layer function $f_i(\overrightarrow{x})$ as follows:

$\overrightarrow{y} = f_i(\overrightarrow{x}; \mathbf{W_i}, \overrightarrow{b_i}; n, m, \sigma_i) = \sigma_i(\mathbf{W_i}\overrightarrow{x} + \overrightarrow{b_i})$

$\mathbf{W_i}$ and $\overrightarrow{b_i}$ are trainable parameters known as weights and biases respectively. $n$, $m$, and $\sigma_i$ are non-trainable hyperparameters referring to the input size, output size, and activation functions respectively.

We define a matrix of weights $\mathbf{W_i}$ for the layer function. The matrix looks like this:

$$
\begin{bmatrix}
w_{11} & w_{21} & ... & w_{n1}\\
w_{12} & w_{22} & ... & w_{n2}\\
\vdots&\vdots&\ddots&\vdots\\
w_{1m} & w_{2m} & ... & w_{nm}\\
\end{bmatrix}
$$

Each row of the matrix corresponds to the weights of 1 neuron. $n$ refers to the size of the input ($\overrightarrow{x}$) and $m$ refers to the size of the output ($\overrightarrow{y}$). The bias vector looks like this:

$$
\begin{bmatrix}
b_{1}\\
b_{2}\\
\vdots\\
b_{m}\\
\end{bmatrix}
$$

We apply the layer function in a chain: we pass the input to the first layer $f_1(\overrightarrow{x})$. The output of this function is passed to the second layer ($f_2(\overrightarrow{x})$. It follows that the output size for layer 1 is the input size of layer 2, the output size of layer 2 is the input size of layer 3, and so on. All in all, the output of the network after the full forward propagation process is $f_N(f_{N-1}(\dots f_2(f_1(\overrightarrow{x}))))$.

## Backpropagation

For backpropagation, the error metric calculated using the output of the neural network is best interpreted as a function of the weights and biases.

The backpropagation function takes the error (our "y value"), and current weights (our "x value") as input, and spits out updated weights (a set of values closer to our desired "x value". We can't find the exact "x value" that makes our error 0 in just 1 step, or probably ever, given that the size of our input is so large). Recall our forward propagation function:

$f_i(\overrightarrow{x}; \mathbf{W_i}, \overrightarrow{b_i}; n, m, \sigma_i) = \sigma_i(\mathbf{W_i}\overrightarrow{x} + \overrightarrow{b_i})$

We can rewrite in terms of our weights:

$f_i(\mathbf{W_i}; \overrightarrow{b_i}; n, m, \overrightarrow{x}, \sigma_i) = \sigma_i(\mathbf{W_i}\overrightarrow{x} + \overrightarrow{b_i})$

### Output Layer backpropagation

We want to minimize the error $E$ by shifting the weights $\mathbf{W_i}, \overrightarrow{b_i}$. That is, we want to know, for what change in $\mathbf{W_i}$ and $\overrightarrow{b_i}$ will $E$ decrease? We look at each separately. 

### $\mathbf{W_N}$

To get the necessary change in $\mathbf{W_N}$, we first need the gradient of $E$ with respect to $\mathbf{W_{N}}$, where $N$ is the last (output) layer. The gradient for our function is as follows, using the chain rule:

Let $g_i(\mathbf{W_i}) = \mathbf{W_i}\overrightarrow{x} + \overrightarrow{b_i}$. Then, $f_i = \sigma_i(g_i(\mathbf{W_i}))$. 

Recall that for our output layer, $E(\overrightarrow{e}, \overrightarrow{a}) = E(f_i(\mathbf{W_i}), \overrightarrow{a}) = E(\sigma_i(g_i(\mathbf{W_i})), \overrightarrow{a})$.

Now, let's assume $\overrightarrow{a}$ is given, then we can rewrite $E$ as $E = E(\sigma_i(g_i(\mathbf{W_i})))$. 

If follows that: $\frac{\partial E}{\partial \mathbf{W_{N}}} = E'(\sigma_i(g_i(\mathbf{W_i}))\sigma'_i(g_i(\mathbf{W_i}))g'_i(\mathbf{W_i})$.

### $\overrightarrow{b_N}$

This follows the exact same process as with $\mathbf{W_{N}}$, though we take the gradient with respect to $\overrightarrow{b_{N}}$. 

Let $g_i(\overrightarrow{b_i}) = \mathbf{W_i}\overrightarrow{x} + \overrightarrow{b_i}$. Then, $f_i = \sigma_i(g_i(\overrightarrow{b_i}))$. 

Recall that for our output layer, $E(\overrightarrow{e}, \overrightarrow{a}) = E(f_N(\overrightarrow{b_N}), \overrightarrow{a}) = E(\sigma_N(g_N(\overrightarrow{b_N})), \overrightarrow{a})$.

Now, let's assume $\overrightarrow{a}$ is given, then we can rewrite $E$ as $E = E(\sigma_N(g_N(\overrightarrow{b_N})))$. 

If follows that: $\frac{\partial E}{\partial \overrightarrow{b_N}} = E'(\sigma_i(g_N(\overrightarrow{b_N}))\sigma'_N(g_N(\overrightarrow{b_N}))g'_N(\overrightarrow{b_N})$.

### Hidden layer backpropagation

Remember that at its very core, a neural network is just a composition of functions $f_{N}(f_{N-1}(...(f_1(\overrightarrow{x}))))$. So if we look at layer $N-1$, we get the following equation with respect to the error:

$E(f_N(\overrightarrow{x}), \overrightarrow{a}) = E(f_N(f_{N-1}(\overrightarrow{z})), \overrightarrow{a})$ where $\overrightarrow{z}$ is the input to layer $N-1$.

### $\mathbf{W_{N-1}}$

Recall that for our output layer, $E(\overrightarrow{e}, \overrightarrow{a}) = E(f_{N}(\mathbf{W_{N}}), \overrightarrow{a}) = E(\sigma_{N}(g_{N}(\mathbf{W_{N}})), \overrightarrow{a})$.

We can rewrite this a bit knowing that our neural network is just a composition of functions. Note that we now write $f_{N}, g_{N}$ as a function of $\overrightarrow{x}$ again, but we write $f_{N-1}$ as a function of $\mathbf{W_{N-1}}$, and we can assume that $\overrightarrow{a}$ is given.

$E(\overrightarrow{e}, \overrightarrow{a}) = E(f_{N}(f_{N-1}(\mathbf{W_{N-1}})), \overrightarrow{a}) = E(\sigma_{N}(g_{N}(\sigma_{N-1}(g_{N-1}(\mathbf{W_{N-1}})))), \overrightarrow{a}) = E(\sigma_{N}(g_{N}(\sigma_{N-1}(g_{N-1}(\mathbf{W_{N-1}})))))$.

Now, we have related the error we got for the output layer to the weights of the first hidden layer. We can now essentially repeat the same process that we did for the output layer, chain rule and all (though the gradients are much less fun to compute).

$\frac{\partial E}{\partial \mathbf{W_{N-1}}} = E'(\sigma_{N}(g_{N}(\sigma_{N-1}(g_{N-1}(\mathbf{W_{N-1}})))))\sigma'_{N}(g_{N}(\sigma_{N-1}(g_{N-1}(\mathbf{W_{N-1}}))))g'_{N}(\sigma_{N-1}(g_{N-1}(\mathbf{W_{N-1}})))\sigma'_{N-1}(g_{N-1}(\mathbf{W_{N-1}}))g'_{N-1}(\mathbf{W_{N-1}})$.

We now need to do the same for the bias vector.

### $\overrightarrow{b_{N-1}}$

$E(\overrightarrow{e}, \overrightarrow{a}) = E(f_{N}(f_{N-1}(\overrightarrow{b_{N-1}})), \overrightarrow{a}) = E(\sigma_{N}(g_{N}(\sigma_{N-1}(g_{N-1}(\overrightarrow{b_{N-1}})))), \overrightarrow{a}) = E(\sigma_{N}(g_{N}(\sigma_{N-1}(g_{N-1}(\overrightarrow{b_{N-1}})))))$.

Now, we have related the error we got for the output layer to the biases of layer $N-1$. We can now essentially repeat the same process that we did for the output layer, chain rule and all (though the gradients are much less fun to compute).

$\frac{\partial E}{\partial \overrightarrow{b_{N-1}}} = E'(\sigma_{N}(g_{N}(\sigma_{N-1}(g_{N-1}(\overrightarrow{b_{N-1}})))))\sigma'_{N}(g_{N}(\sigma_{N-1}(g_{N-1}(\overrightarrow{b_{N-1}}))))g'_{N}(\sigma_{N-1}(g_{N-1}(\overrightarrow{b_{N-1}})))\sigma'_{N-1}(g_{N-1}(\overrightarrow{b_{N-1}}))g'_{N-1}(\overrightarrow{b_{N-1}})$.

### Backpropagation after layer $N-1$

The process works the same. We first update the weights $\mathbf{W_i}$. For any layer $i$, we have 

$g_i(\mathbf{W_i}) = \mathbf{W_i}\overrightarrow{x} + \overrightarrow{b_i}$. Then, $f_i = \sigma_i(g_i(\mathbf{W_i}))$. 

$E(\overrightarrow{e}, \overrightarrow{a}) = E(f_N(f_{N-1}(\dots f_i(\mathbf{W_i}))), \overrightarrow{a}) = E(\sigma_N(g_N(\sigma_{N-1}(g_{N-1}(\dots \sigma_i(g_i(\mathbf{W_i}))))), \overrightarrow{a})$.

The gradient is 

$$\frac{\partial E}{\partial \mathbf{W_i}} = $$

$$E'(\sigma_N(g_N(\sigma_{N-1}(g_{N-1}(\dots \sigma_i(g_i(\mathbf{W_i})))))$$

$$\sigma'_{N}(g_{N}(\sigma_{N-1}(g_{N-1}(\dots \sigma_i(g_i(\mathbf{W_i}))))))$$

$$g'_{N}(\sigma_{N-1}(g_{N-1}(\dots \sigma_i(g_i(\mathbf{W_i})))))$$

$$\sigma'_{N-1}(g_{N-1}(\dots \sigma_i(g_i(\mathbf{W_i}))))$$

$$g'_{N-1}(\dots \sigma_i(g_i(\mathbf{W_i})))$$

$$\dots \sigma'_i(g_i(\mathbf{W_i}))g'_i(\mathbf{W_i})$$.

We now update the biases $\overrightarrow{b_i}$. For any layer $i$, we have 

$g_i(\overrightarrow{b_i}) = \mathbf{W_i}\overrightarrow{x} + \overrightarrow{b_i}$. Then, $f_i = \sigma_i(g_i(\overrightarrow{b_i}))$. 

$E(\overrightarrow{e}, \overrightarrow{a}) = E(f_N(f_{N-1}(\dots f_i(\overrightarrow{b_i}))), \overrightarrow{a}) = E(\sigma_N(g_N(\sigma_{N-1}(g_{N-1}(\dots \sigma_i(g_i(\overrightarrow{b_i}))))), \overrightarrow{a})$.

The gradient is 

$$\frac{\partial E}{\partial \overrightarrow{b_i}} = $$

$$E'(\sigma_N(g_N(\sigma_{N-1}(g_{N-1}(\dots \sigma_i(g_i(\overrightarrow{b_i})))))$$

$$\sigma'_{N}(g_{N}(\sigma_{N-1}(g_{N-1}(\dots \sigma_i(g_i(\overrightarrow{b_i}))))))$$

$$g'_{N}(\sigma_{N-1}(g_{N-1}(\dots \sigma_i(g_i(\overrightarrow{b_i})))))$$

$$\sigma'_{N-1}(g_{N-1}(\dots \sigma_i(g_i(\overrightarrow{b_i}))))$$

$$g'_{N-1}(\dots \sigma_i(g_i(\overrightarrow{b_i})))$$

$$\dots \sigma'_i(g_i(\overrightarrow{b_i}))g'_i(\overrightarrow{b_i})$$.

### Updating the weights and biases

Once we have the gradient, updating the weights and biases is very simple.

$\mathbf{W_i} = \mathbf{W_i} - l \cdot \frac{\partial E}{\partial \mathbf{W_i}}\overrightarrow{1_m^T}$ (since the partial derivative is a vector)

$\overrightarrow{b_i} = \overrightarrow{b_i} - l \cdot \frac{\partial E}{\partial \overrightarrow{b_i}}$

## Do these equations translate directly to code?

Mostly. There's some nuances though. It's easy for notational purposes to write the equations the way we did but there's some extra matrix/vector transposition to make sure the dimensions line up. See the code below.

## Implementation

### *Sidenote: does it work?*

The answer is mostly not. The sample network is set up to solve the iris dataset- given 4 attributes of a flower it needs to classify the flower into 1 of 3 categories. It does not output the correct prediction- it outputs the probability of getting a given type of flower. I'm not sure why it does that. There's probably a typo somewhere.

In [ ]:
import numpy as np
from numpy.random import default_rng
import math
import statistics
from sklearn import datasets
from sklearn.utils import shuffle
from tqdm import tqdm

iris = datasets.load_iris()

In [ ]:
class Layer:
    def __init__(self, n, m, lr, activation, derivative, loss, loss_derivative, next=None, prev=None, batch_size=32):
        self.n = n
        self.m = m
        self.lr = lr
        self.activation = activation
        self.activation_derivative = derivative
        self.loss = loss
        self.loss_derivative = loss_derivative
        self.next = next
        self.prev = prev
        
        self.W = np.matrix(np.random.normal(loc=0, scale=1/n, size=n * m).reshape(m, n))
        self.b = np.zeros(m)

        self.inputs = []
        self.outputs = []
        
        self.batch_size = batch_size

    def forward_propagation(self, input):
        self.inputs.append(input)
        self.outputs.append(np.asarray(np.matmul(self.W, input) + self.b).reshape(-1))
        if self.next:
            return self.next.forward_propagation(self.activation(self.outputs[-1]), bn=bn)
        else:
            return self.activation(self.outputs[-1])

    def backpropagation(self, expected_outputs=None, delta_list=None):
        if len(self.outputs) < self.batch_size:
            return False
        bias_update = np.zeros(self.m).reshape(self.m,)
        weight_update = np.zeros(self.n * self.m).reshape(self.m, self.n)
        deltas = []
        if not self.next:
            if expected_outputs is None:
                raise ValueError("error not given to output layer")
            for input, output, result in zip(self.inputs, self.outputs, expected_outputs):
                inp = input.reshape(-1, 1)
                loss_d = np.dot(self.loss_derivative(self.activation(output), result), self.activation_derivative(output)).reshape(-1, 1)
                bias_update += loss_d.reshape(-1)
                weight_update += np.dot(loss_d, inp.T)
                deltas.append(np.asarray(np.dot(self.W.T, loss_d.reshape(-1))).reshape(-1))
            if self.prev:
                ret = self.prev.backpropagation(delta_list=deltas)
            else:
                ret = True
        else:
            if deltas is None:
                raise ValueError("delta not given to hidden layer")
            for input, output, delta in zip(self.inputs, self.outputs, delta_list):
                inp = input.reshape(-1, 1)
                dt = delta.reshape(1, -1)
                weight_delta = (delta * self.activation_derivative(output)).reshape(-1) # size m
                bias_update += weight_delta
                weight_update += np.dot(weight_delta.reshape(-1, 1), inp.T)
                deltas.append(np.asarray(np.dot(self.W.T, weight_delta)).reshape(-1))
            if self.prev:
                ret = self.prev.backpropagation(delta_list=deltas)
            else:
                ret = True
        
        self.W -= self.lr / self.batch_size * weight_update
        self.b += self.lr / self.batch_size * bias_update

        self.inputs = []
        self.outputs = []

        return ret

In [ ]:
np.random.seed(seed=47)

# building training set
X_tr, y_tr = shuffle(iris.data, iris.target)

X_train = []
y_train = []

for i in range(len(y_tr)):
    X_tr[i] -= statistics.mean(X_tr[i])
    X_tr[i] /= statistics.stdev(X_tr[i])
    
    X_train.append(X_tr[i])
    y_train.append([0.98 if y_tr[i] == j else 0.01 for j in range(3)])

# activation functions
sigmoid = lambda x: np.array([1 / (1 + math.exp(i)) for i in np.clip(x, -500, 500)])
d_sigmoid = lambda x: sigmoid(x) * (np.ones(len(x)) - sigmoid(x))

tanh = lambda x: np.array([(math.exp(i) - math.exp(-i)) / (math.exp(i) + math.exp(-i)) for i in x])
d_tanh = lambda x: np.ones(len(x)) - tanh(x) ** 2

relu = lambda x: np.array(list(map(lambda y: y if y > 0 else 0, x)))
d_relu = lambda x: np.array(list(map(lambda y: 1 if y > 0 else 0, x)))

softmax = lambda x: np.exp(np.clip(x, -500, 500)) / np.sum(np.exp(np.clip(x, -500, 500)))

def d_softmax(x):
    s = np.clip(x, -500, 500).reshape(-1,1)
    return np.diagflat(s) - np.dot(s, s.T)

# loss functions
mse = lambda x, y: np.array([(a - b) ** 2 for a, b in zip(x, y)])
d_mse = lambda x, y: np.array([-2 * (a - b) for a, b in zip(y, x)])

cce = lambda x, y: -np.sum([b * np.log(a + 10 ** -100) for a, b in zip(x, y)])
d_cce = lambda x, y: np.array([-b / (a + 10 ** -100) + (1 - b) / (1 - a + 10 ** -100) for a, b in zip(x, y)])

# hyperparameters
lr = 0.5
num_epochs = 500
bs = 2

# building network
l1 = Layer(4, 50, lr, sigmoid, d_sigmoid, mse, d_mse, batch_size=bs)
l2 = Layer(50, 20, lr, sigmoid, d_sigmoid, mse, d_mse, batch_size=bs)
l3 = Layer(20, 10, lr, sigmoid, d_sigmoid, mse, d_mse, batch_size=bs)
l4 = Layer(10, 3, lr, softmax, d_softmax, cce, d_cce, batch_size=bs)

l1.next = l2
l2.prev = l1
l2.next = l3
l3.prev = l2
l3.next = l4
l4.prev = l3

nn = [l1, l2, l3, l4]

# training loop
outputs_buffer = []

for i in range(num_epochs):
    losses = []
    print(f"Epoch: {i+1}")

    trip = False

    for X, y in tqdm(zip(X_train[:10000], y_train[:10000])):
        pred = nn[0].forward_propagation(X)
        try:
            losses.append(statistics.mean(nn[-1].loss(pred, y)))
        except:
            losses.append(nn[-1].loss(pred, y))
            if losses[-1] > 3: trip = True
        outputs_buffer.append(y)
        successful = nn[-1].backpropagation(expected_outputs=outputs_buffer)
        if successful:
            outputs_buffer = []
        if trip:
            break
    if trip:
        break
    print(f"Average loss: {statistics.mean(losses)}")

In [ ]:
np.set_printoptions(formatter={'float': lambda x: "{0:0.3f}".format(x)})

for i in range(45, 50):
    print(nn[0].forward_propagation(X_train[i]), y_train[i])
    print(nn[-1].loss(nn[0].forward_propagation(X_train[i]), y_train[i]))
    print(nn[-1].loss_derivative(nn[0].forward_propagation(X_train[i]), y_train[i]))